In [ ]:
from soss_tfit.core import core
from soss_tfit.science import mcmc
from soss_tfit.science import plot

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [14.0, 6.0]

# printer
cprint = core.base_classes.Printer()

In [ ]:
#load the Sampler object from pickle file
# pfile="/home/dlafre/projects/def-dlafre/dlafre/tfit/outputs/wasp-52b_t_logg_sampler_trial.pickle"
pfile="/home/dlafre/projects/def-dlafre/dlafre/tfit/outputs/wasp-52b_fixed_ld_sampler_trial.pickle"
sampler=mcmc.Sampler.load(pfile)
tfit=sampler.tfit


#some info
print('Mode:',sampler.mode)
print('Number of walkers:',len(sampler.wchains))
print('Number of loops completed:',sampler.nloop)
print('Number of steps completed by each walker:',sampler.wchains[0].shape[0]*tfit.params['THINNING'])
print()
print('Number of steps/walker for next loop:',sampler.nsteps)
print('Increment of steps per loop:',tfit.params['NSTEPS_INC']['full'])
print()
print('Number of photometric bins',tfit.n_phot)
print('Number of fitted paramters',tfit.x0.size)

In [ ]:
#show the model parameters and their value
bpass=55
for i,n in enumerate(tfit.pnames):
    print(i,n,tfit.p0[i,bpass])

In [ ]:
#show model fit
bpass=144
plot.plot_transit_fit(tfit,bpass,residuals=1)

In [ ]:
#plot fit for all spectroscopic ligth curves
plot.plot_transit_fit(tfit)

In [ ]:
# compile results
cprint('Compiling results [percentiles]')
result_table = sampler.results(start_chain=10)

cprint('Results [percentile]:')
sampler.print_results('percentile')

In [ ]:
#plot the transmission spectrum
fig,ax = plot.plot_rprs_sq(tfit, sampler.results_table,return_object=True)
ax.set_xscale('log')

In [ ]:
#show the walker chains
ikey=730
ikey,=(tfit.xnames=="SMA1").nonzero()
# bpass=144
# ikey=ikey[bpass]

print(tfit.xnames[ikey])
plt.figure(figsize=(14,5))
for w in range(len(sampler.wchains)):
    plt.plot(sampler.wchains[w][:,ikey])

In [ ]:
#sampler loglikelihood
for w in range(len(sampler.wlls)):
    plt.plot(sampler.wlls[w])

In [ ]:
#plot the spectrum of any chromatic parameter
k="TC1"
fig,ax = plot.plot_spectrum(tfit, sampler.results_table,
                   key=k,plot_label=k,pkind='percentile',return_object=True)

In [ ]:
#if wanted, use best solution to mask outlier points
do_mask=True
nsig=3

i_dsc,=(tfit.pnames=="DSC").nonzero()[0]
#loop through band passes, identify outliers, amplify their error (same effect as masking)
for bpass in range(tfit.n_phot):
    model=tfit.tmodel_func(tfit,bpass)
    res=np.abs(tfit.flux[bpass]-model)
    #compared to original errors
    # res_sig=res/np.nanmedian(tfit.fluxerr[bpass])
    #compared to scaled errors
    res_sig=res/np.nanmedian(tfit.fluxerr[bpass]*tfit.p0[i_dsc,bpass])
    
    ibad,=(res_sig>nsig).nonzero()
    print(bpass,tfit.wavelength[0,bpass],tfit.p0[i_dsc,bpass],'N outliers=',ibad.size)
    if do_mask:
        tfit.fluxerr[bpass,ibad]=np.nan

In [ ]:
#dump sampler again to save its updated tfit
sampler.dump()

In [ ]:
tfit.params['NSTEPS']['full']=50000

In [ ]:
sampler.params['NSTEPS']